## Code for tickbox checked detection

In [1]:
# import the necessary packages
import numpy as np
import imutils
from imutils import contours
import cv2
# OS module provides functions that can interact with the operating system
import os
import glob


### Step 1 Get Images & Pre processing -------------------------------------------------------------------------------------
###Create image directory, to load in multiple images and pre processing steps (resize,crop, gray scale, blur, edges)

# create an image directory (adding r to front of string converts string to a raw string)
img_dir = (r" ") # enter file pathway here
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)

# initalise lists to append
# RESULTS KEY: 1 = positive, 2 = negative, 3 = inconclusive, 4 = code failed to detect result/ no result filled in
images = []
tickBoxIm = []
results =[]
#debug = []
for f in files:
    im = cv2.imread(f)
    # resize image, ensures all images are same size
    ratio = 400.0 / im.shape[1]
    dimensions = (400, int(im.shape[0] * ratio))
    im = cv2.resize(im, dimensions, interpolation = cv2.INTER_AREA)
    
    # crop image, change to grey scale and then apply blurring filter before using Canny edge detection
    im_crop = im[(1*int(im.shape[0]/5)):4*(int(im.shape[0]/6)), int((im.shape[1]/2)):(int(im.shape[1]))]
    gray = cv2.cvtColor(im_crop, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    im_canny = cv2.Canny(blurred, 75, 255)
    images.append(im_canny)
    
### Step 2 Get tickboxes ----------------------------------------------------------------------------------------------------    
### Binnarise image & use Otsus thresholding, then get and set contour parameters to detect tickboxes   

    for i in images:
        thresh = cv2.threshold(i, 0, 255,cv2.THRESH_BINARY,cv2.THRESH_OTSU)[1]
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        #print(len(cnts))
        cnts = imutils.grab_contours(cnts)
        # initalise empty tick box lsit
        tickBox = []        
        # loop over the contours
        for c in cnts:
            # get bounding box, so can use width and height to detect boxes
            (x, y, w, h) = cv2.boundingRect(c)
            area = cv2.contourArea(c)
            # Aspect ratio use? 
            # ar = w / float(h)
            # bounding parameters
            if w >= 15 and w <= 40 and h >= 15 and h <= 35 and area > 225: #ar >= 0.6 and ar <= 1.4:
                tickBox.append(c)
                #print(len(tickBox))
                box = cv2.drawContours(i,tickBox,-1,(255,0,0),3)
                
# ATTEMPTS TO FORCE RETURN TO NEXT IMAGE IF 3 TICKBOXES NOT DETECTED ######################################################
#                 if len(tickBox) != 3:
#                     results.append(4)
#                     break
#                 else:
#                     box = cv2.drawContours(i,tickBox,-1,(255,0,0),3)
############################################################################################################################
            
                #cv2.imshow('',box)
                #cv2.waitKey(0)
    tickBoxIm.append(box)
    
### Step 3 Extract and Return Result --------------------------------------------------------------------------------------- 
    
    # each image has three possible 'answers'
    for (q, i) in enumerate(np.arange(0, len(tickBox), 3)):
        # SORTING CONTOURS TO DEAL WITH LANDSCAPE OR PORTRAIT PHOTOS???
        tickBox = contours.sort_contours(tickBox,method="top-to-bottom")[0]    
        # loop over cintiurs applying a mask each time and counting non zero pixels
        pixCountIndex = []
        for (index, c) in enumerate(tickBox):
            #tickBox = contours.sort_contours(tickBox, method="left-to-right")[0]
            mask = np.zeros(box.shape, dtype="uint8")
            cv2.drawContours(mask, [c], -1, 255, -1)
            #cv2.imshow('mask',mask)
            #cv2.waitKey(0)
            mask = cv2.bitwise_and(thresh, thresh, mask=mask)
            total = cv2.countNonZero(mask)
            #print(total, index)
            pixCountIndex.append(total) 
            pixCountIndex.append(index)
            #print(pixCountIndex)
            #print(len(pixCountIndex))
        # once finished looping works through a series of if statements
        maxVal = max(pixCountIndex)
        no_mark_low = maxVal * 0.9
        no_mark_high = maxVal * 1.1 
#         if (pixCountIndex[0] == ''):
#             results.append(4)
#             print('here2')
#             break
        if (len(pixCountIndex) != 6):
            results.append(4)
            #print('here1')
            break
        if (pixCountIndex[0] == maxVal) & (pixCountIndex[1] == 0) & (pixCountIndex[0] >= (1.1* pixCountIndex[2])) & (pixCountIndex[0] >= (1.1* pixCountIndex[4])):
            results.append(1)
        if (pixCountIndex[2] == maxVal) & (pixCountIndex[3] == 1) & (pixCountIndex[2] >= (1.1* pixCountIndex[0])) & (pixCountIndex[2] >= (1.1* pixCountIndex[4])):
            results.append(2)
        if (pixCountIndex[4] == maxVal) & (pixCountIndex[5] == 2)& (pixCountIndex[4] >= (1.1* pixCountIndex[0])) & (pixCountIndex[4] >= (1.1* pixCountIndex[2])):
            results.append(3)
        if (no_mark_low <= pixCountIndex[0] <= no_mark_high) & (no_mark_low <= pixCountIndex[2] <= no_mark_high) & (no_mark_low <= pixCountIndex[4] <= no_mark_high):
            results.append(4)            
            #results.append(i)
        #print(results)
# cv2.imshow('test',images[5])
# cv2.waitKey(0)

In [2]:
# len(results)
#results

58